## 搭建单个神经元进行训练

In [1]:
import pickle
import os
import numpy as np
CIFAR_DIR = "../data/cifar-10-batches-py"

In [2]:
for file in os.listdir(CIFAR_DIR):
    print(file.title())
print(os.path.join(CIFAR_DIR,os.listdir(CIFAR_DIR)[1].title()))

Batches.Meta
Data_Batch_1
Data_Batch_2
Data_Batch_3
Data_Batch_4
Data_Batch_5
Readme.Html
Test_Batch
../data/cifar-10-batches-py\Data_Batch_1


In [3]:
with open(os.path.join(CIFAR_DIR,os.listdir(CIFAR_DIR)[1].title()),"rb") as ff:
    data = pickle.load(ff,encoding="bytes")
    print(data.keys())

dict_keys([b'batch_label', b'labels', b'data', b'filenames'])


In [4]:
def load_data(filename):
    """read data from data file"""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding = 'bytes')
        return data[b'data'], data[b'labels']
class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        """从文件中读取cifar数据"""
        for filename in filenames:
            f_data, f_labels = load_data(filename)
            for data, label in zip(f_data, f_labels):
                if label in [0, 1]:
                    all_data.append(data)
                    all_labels.append(label)       
        self._data = np.vstack(all_data)
        # 进行归一化处理
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        print(self._data.shape, self._labels.shape)
        self._num_examples = len(self._data)
        self._indicator = 0
        self._need_shuffle = need_shuffle
        if need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        index = np.random.permutation(self._num_examples)
        self._data = self._data[index]
        self._labels = self._labels[index]
    
    def next_batch(self, batch_size):
        if batch_size > self._num_examples:
            raise Exception("the batch size is lager than all examples")
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels
train_filenames = [os.path.join(CIFAR_DIR, "Data_Batch_%d" % i)  
                   for i in range(1, 6)]
test_filename = [os.path.join(CIFAR_DIR, "Test_Batch")]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filename, False)

# batch_data, batch_labels = train_data.next_batch(10)
# print(batch_data)
# print(batch_labels)

(10000, 3072) (10000,)
(2000, 3072) (2000,)


In [5]:
from tensorflow import compat as tf_compat
tf = tf_compat.v1
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


## 搭建单个神经元

In [6]:
x = tf.placeholder(tf.float32, [None, 3072]) # 输入神经元的值
y = tf.placeholder(tf.int64,[None]) # 输出标签

# 每一个神经元输入的权重
w = tf.get_variable('w', [x.get_shape()[-1], 1], 
                    initializer = tf.random_normal_initializer(0, 1))
# 偏置节点的值
b = tf.get_variable('b', [1], initializer = tf.constant_initializer(0.0))
# 将输入的值经过权值处理后
y_ = tf.matmul(x, w) + b
# 将输入的值进行sigmoid处理
y_p_1 = tf.nn.sigmoid(y_)
# 将y值进行reshape处理
y_reshape = tf.reshape(y, [-1, 1])
# 转换int为float类型                                             
y_reshape_float = tf.cast(y_reshape, tf.float32)
loss = tf.reduce_mean(tf.square(y_reshape_float - y_p_1))

predict = y_p_1 > 0.5
correct_prediction = tf.equal(tf.cast(predict, tf.int64), y_reshape)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [8]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100
with tf.Session() as sess:
    # init
    sess.run(init)
    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        loss_val, acc_val, _ = sess.run(
            [loss, accuracy, train_op], 
            feed_dict = {
                x: batch_data,
                y: batch_labels
            })
        # print training log
        if (i + 1) % 500 == 0:
            print("[Train] Step %d, loss: %4.5f, acc: %4.5f"  
                  % (i + 1, loss_val, acc_val))
        # test the model
        if (i + 1) % 2000 == 0:
            test_data = CifarData(test_filename, False)
            test_acc_all = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy], 
                    feed_dict = {
                        x: test_batch_data,
                        y: test_batch_labels
                    })
                test_acc_all.append(test_acc_val)
            test_acc = np.mean(test_acc_all)
            print("[Test] Step: %d, acc: %4.5f" % (i + 1, test_acc ))

[Train] Step 500, loss: 0.24872, acc: 0.75000
[Train] Step 1000, loss: 0.15005, acc: 0.85000
[Train] Step 1500, loss: 0.26409, acc: 0.70000
[Train] Step 2000, loss: 0.24489, acc: 0.75000
(2000, 3072) (2000,)
[Test] Step: 2000, acc: 0.76050
[Train] Step 2500, loss: 0.15000, acc: 0.85000
[Train] Step 3000, loss: 0.25000, acc: 0.75000
[Train] Step 3500, loss: 0.34950, acc: 0.65000
[Train] Step 4000, loss: 0.05000, acc: 0.95000
(2000, 3072) (2000,)
[Test] Step: 4000, acc: 0.79050
[Train] Step 4500, loss: 0.35090, acc: 0.65000
[Train] Step 5000, loss: 0.34490, acc: 0.65000
[Train] Step 5500, loss: 0.11056, acc: 0.90000
[Train] Step 6000, loss: 0.29645, acc: 0.70000
(2000, 3072) (2000,)
[Test] Step: 6000, acc: 0.79700
[Train] Step 6500, loss: 0.38203, acc: 0.60000
[Train] Step 7000, loss: 0.20231, acc: 0.80000
[Train] Step 7500, loss: 0.14979, acc: 0.85000
[Train] Step 8000, loss: 0.19981, acc: 0.80000
(2000, 3072) (2000,)
[Test] Step: 8000, acc: 0.80100
[Train] Step 8500, loss: 0.27670, acc